In [ ]:
import pandas as pd
import spotipy
#from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
df = pd.read_csv('global_2017-07-14--2017-07-21.csv')

In [ ]:
df

In [ ]:
df.columns = df.iloc[0]

In [ ]:
df = df.iloc[1:].reset_index()

In [ ]:
df.URL.astype('str')

In [ ]:
df.URL = df.URL.apply(lambda x : x[31:])

In [ ]:
df.rename(columns={'URL':'ID'}, inplace=True)

In [ ]:
df

In [ ]:
credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()

spotify = spotipy.Spotify(auth=token)

features = spotify.audio_features(str(df['ID'].iloc[0]))


In [ ]:
features

In [ ]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [ ]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [ ]:
prova = extract_features(df, n_tracks=200)

In [ ]:
prova

In [ ]:
output = df.join(prova.set_index('id'), on='ID')

In [ ]:
output.to_csv('global_08-2017.csv')

In [ ]:
output

test=output.iloc[9]
print(test)
print(type(test['Track Name']))
if type(test['Track Name']) != float and type(test['Artist']) != float:
    print('no empty')
else: 
    print('empty')

## Testing iTunes API for genre labeling
Simple test of quering on iTunes API

In [ ]:
import requests
import json
entry = 'post+malone+circles'
a = 'https://itunes.apple.com/search?term='+entry+'.&limit=1'
b = requests.get(a).json()
print(b['results'][0]['primaryGenreName'])
print(json.dumps(b, indent=2))

# Automatizing the feature extraction process

In [2]:
import pandas as pd
import spotipy
#from fycharts.SpotifyCharts import SpotifyCharts
from spotipy.oauth2 import SpotifyClientCredentials
import os
import requests
import json
import time

In [3]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        print(features[0]['danceability'])
        dataframe.append({'danceability' : features[0]['danceability'],
        'energy': features[0]['energy'],
        'key' : features[0]['key'],
        'loudness' : features[0]['loudness'],
        'mode' : features[0]['mode'],
        'speechiness' : features[0]['speechiness'],
        'acousticness' : features[0]['acousticness'],
        'instrumentalness' : features[0]['instrumentalness'],
        'liveness' : features[0]['liveness'],
        'valence' : features[0]['valence'],
        'tempo' : features[0]['tempo'],
        'time_signature' : features[0]['time_signature'],
        'duration_ms' : features[0]['duration_ms']}, ignore_index=True)
    return dataframe

In [4]:
def extract_features(df, n_tracks = 50):
    dataframe = pd.DataFrame(columns=['id','danceability','energy','key','loudness','mode','speechiness',
                                     'acousticness','instrumentalness','liveness','valence','tempo','time_signature',
                                     'duration_ms'])
    for i in range(n_tracks):
        features = spotify.audio_features(str(df['ID'].iloc[i]))
        dataframe.loc[i] = [ features[0]['id'],
        features[0]['danceability'],
        features[0]['energy'],
        features[0]['key'],
        features[0]['loudness'],
        features[0]['mode'],
        features[0]['speechiness'],
        features[0]['acousticness'],
        features[0]['instrumentalness'],
        features[0]['liveness'],
        features[0]['valence'],
        features[0]['tempo'],
        features[0]['time_signature'],
        features[0]['duration_ms']]
    return dataframe

In [5]:
#checking data integrity. Checks for missing fields in Artist name. Return the rows with missing values + the file name. 
#These data are missing because of copyright issues. They changed the author of the song/removed the song. 
import numpy as np

from IPython.display import display, HTML

path = os.getcwd()
file_folder = os.path.join(path,'gb')

for csv_file in os.listdir(file_folder):
    file = os.path.join(file_folder,csv_file)
    #print(csv_file)
    df = pd.read_csv(file)
    #display(df)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df.URL.astype('str')
    df.URL = df.URL.apply(lambda x : x[31:])
    df.rename(columns={'URL':'ID'}, inplace=True)
    
    res = df.loc[df['Artist'].isnull(), 'index'].tolist()
    if len(res) != 0:
        print(res)
        print(csv_file)

    
    
 

[7, 30, 42, 60]
gb_2017-07-14--2017-07-21.csv


In [7]:
path = os.getcwd()
file_folder = os.path.join(path,'de 2')
save_folder_path = os.path.join(path, "de_features")
#os.mkdir(save_folder_path)

for csv_file in os.listdir(file_folder): 
    print(csv_file)
    file = os.path.join(file_folder,csv_file)
    df = pd.read_csv(file)
    df.columns = df.iloc[0]
    df = df.iloc[1:].reset_index()
    df.URL.astype('str')
    df.URL = df.URL.apply(lambda x : x[31:])
    df.rename(columns={'URL':'ID'}, inplace=True)
    credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
    token = credentials.get_access_token()
    spotify = spotipy.Spotify(auth=token)
    features = spotify.audio_features(str(df['ID'].iloc[0]))
    prova = extract_features(df, n_tracks=200)
    output = df.join(prova.set_index('id'), on='ID')
    output['genre']= "none"
    for index, row in output.iterrows():
        artist = row["Artist"]
        song = row["Track Name"]
        if type(artist) != float and type(song) != float:
            song = song.split('(')[0]
            query_entry = str(artist)+" "+str(song)
            query_entry = query_entry.replace("with", "")
            query_entry= query_entry.replace('Remastered', "")
            query_entry= query_entry.replace('Version', "")

            query_entry = query_entry.replace("'", "")
            query_entry = query_entry.replace('"', "")

            query_entry=query_entry.replace(" ", "+")

            #query_entry = query_entry[0:60]
            query_entry = query_entry.lower()
            print(query_entry)
            a = 'https://itunes.apple.com/search?term='+query_entry+'&limit=1'
            test = requests.get(a)
            print(test)
            b = test.json()
            if b['resultCount'] == 1:

                print(b['results'][0]['primaryGenreName'])
                #assign genre to song
                print(index)
                output.iloc[index, output.columns.get_loc('genre')]= b['results'][0]['primaryGenreName']
            time.sleep(0.5)
            #print(json.dumps(b, indent=2))
    output.to_csv(os.path.join(save_folder_path,csv_file))




de_2020-04-24--2020-05-01.csv


C:\Users\Mattia\anaconda3\envs\MyEnv\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  app.launch_new_instance()


ufo361+emotions
<Response [200]>
Hip-Hop/Rap
0
shirin+david+90-60-111
<Response [200]>
Hip-Hop/Rap
1
drake+toosie+slide
<Response [200]>
Hip-Hop/Rap
2
the+scotts+the+scotts
<Response [200]>
Hip-Hop/Rap
3
the+weeknd+blinding+lights
<Response [200]>
R&B/Soul
4
saint+jhn+roses+-+imanbek+remix
<Response [200]>
Dance
5
samra+baebae
<Response [200]>
Hip-Hop/Rap
6
ufo361+final+fantasy
<Response [200]>
Hip-Hop/Rap
7
capital+bra+nicht+verdient
<Response [200]>
Hip-Hop/Rap
8
topic+breaking+me
<Response [200]>
Dance
9
surf+mesa+ily+
<Response [200]>
Dance
10
kayef+ich+würd+lügen
<Response [200]>
Pop
11
samra+365+tage
<Response [200]>
Hip-Hop/Rap
12
ufo361+allein+sein
<Response [200]>
Hip-Hop/Rap
13
dardan+wie+lang?????????
<Response [200]>
Rap
14
céline+tränen+aus+kajal
<Response [200]>
Hip-Hop/Rap
15
powfu+death+bed+
<Response [200]>
Hip-Hop/Rap
16
ava+max+salt
<Response [200]>
Pop
17
apache+207+roller
<Response [200]>
Hip-Hop/Rap
18
tones+and+i+dance+monkey
<Response [200]>
Alternative
19
symba

samra+mon+ami
<Response [200]>
Hip-Hop/Rap
162
nimo+kein+schlaf
<Response [200]>
Hip-Hop/Rap
163
the+cratez+honda+civic
<Response [200]>
Rap
164
eno+dämonen
<Response [200]>
Hip-Hop/Rap
165
bhz+flasche+luft
<Response [200]>
Hip-Hop/Rap
166
gzuz+donuts
<Response [200]>
Hip-Hop/Rap
167
black+eyed+peas+ritmo+
<Response [200]>
Pop
168
dardan+genauso
<Response [200]>
Rap
169
vize+never+let+me+down
<Response [200]>
Dance
170
jerome+take+my+hand
<Response [200]>
Electronica
171
kygo+ill+wait
<Response [200]>
Dance
172
dhurata+dora+lass+los
<Response [200]>
Hip-Hop/Rap
173
juice+wrld+lucid+dreams
<Response [200]>
Hip-Hop/Rap
174
felix+jaehn+sicko
<Response [200]>
Dance
175
kontra+k+letzte+träne
<Response [200]>
Hip-Hop/Rap
176
jp+saxe+if+the+world+was+ending+-+feat.+julia+michaels
<Response [200]>
Pop
177
maestro+schrei+
<Response [200]>
Hip-Hop/Rap
178
harry+styles+adore+you
<Response [200]>
Pop
179
dynoro+in+my+mind
<Response [200]>
Dance
180
ufo361+big+drip+-+sonus030+remix
<Response [200]>

jamule+ich+hol+dich+ab
<Response [200]>
Hip-Hop/Rap
123
riton+turn+me+on+
<Response [200]>
Dance
124
blackbear+hot+girl+bummer
<Response [200]>
Pop
125
monet192+dämon+
<Response [200]>
Hip-Hop/Rap
126
vize+thank+you+[not+so+bad]
<Response [200]>
Dance
127
apache+207+kein+problem
<Response [200]>
Hip-Hop/Rap
128
emilio+roter+sand
<Response [200]>
Pop
129
ali+gatie+what+if+i+told+you+that+i+love+you
<Response [200]>
Pop
130
the+weeknd+in+your+eyes
<Response [200]>
R&B/Soul
131
farid+bang+nador+city+gang
<Response [200]>
Hip-Hop/Rap
132
sam+smith+i’m+ready+
<Response [200]>
Pop
133
felix+jaehn+sicko
<Response [200]>
Dance
134
partynextdoor+believe+it
<Response [200]>
R&B/Soul
135
dardan+6am.........
<Response [200]>
Rap
136
sinan-g+action
<Response [200]>
Hip-Hop/Rap
137
jerome+light
<Response [200]>
Hip-Hop/Rap
138
samra+wieder+lila
<Response [200]>
Hip-Hop/Rap
139
ufo361+nur+zur+info
<Response [200]>
Hip-Hop/Rap
140
lena+better
<Response [200]>
Fiction
141
kontra+k+letzte+träne
<Respons

...

<Response [200]>
Pop
120
alicia+keys+underdog
<Response [200]>
Pop
121
sarah+lombardi+te+amo+mi+amor
<Response [200]>
Pop
122
travis+scott+goosebumps
<Response [200]>
Hip-Hop/Rap
123
thedodo+endorphine
<Response [200]>
Trance
124
trevor+daniel+falling
<Response [200]>
Pop
125
apache+207+wieso+tust+du+dir+das+an?
<Response [200]>
Hip-Hop/Rap
126
jack+harlow+whats+poppin+
<Response [200]>
Hip-Hop/Rap
127
pop+smoke+gangstas
<Response [200]>
Hip-Hop/Rap
128
rin+alien
<Response [200]>
Rock
129
manuellsen+2+meter
<Response [200]>
Hip-Hop/Rap
130
shirin+david+90-60-111
<Response [200]>
Hip-Hop/Rap
131
ali+gatie+running+on+my+mind
<Response [200]>
R&B/Soul
132
lewis+capaldi+before+you+go
<Response [200]>
Alternative
133
apache+207+doch+in+der+nacht
<Response [200]>
Hip-Hop/Rap
134
apache+207+kein+problem
<Response [200]>
Hip-Hop/Rap
135
apache+207+matrix
<Response [200]>
Hip-Hop/Rap
136
capital+bra+tilidin
<Response [200]>
Hip-Hop/Rap
137
pop+smoke+aim+for+the+moon+
<Response [200]>
Hip-Hop/Ra

<Response [200]>
Dance
83
juice+wrld+righteous
<Response [200]>
Hip-Hop/Rap
84
sido+2002
<Response [200]>
Hip-Hop/Rap
85
tujamo+lonely
<Response [200]>
Dance
86
ava+max+salt
<Response [200]>
Pop
87
gamper+&+dadoni+gimme!+gimme!+gimme!
<Response [200]>
House
88
luciano+late+night
<Response [200]>
Hip-Hop/Rap
89
maestro+durch+die+nacht
<Response [200]>
Hip-Hop/Rap
90
capital+bra+110
<Response [200]>
Hip-Hop/Rap
91
tom+gregory+fingertips
<Response [200]>
Dance
92
pop+smoke+for+the+night+
<Response [200]>
Hip-Hop/Rap
93
ty+dolla+$ign+ego+death+
<Response [200]>
Hip-Hop/Rap
94
badmómzjay+snowbunny
<Response [200]>
Hip-Hop/Rap
95
felix+jaehn+close+your+eyes
<Response [200]>
Dance
96
monet192+soso+
<Response [200]>
Hip-Hop/Rap
97
pashanim+shababs+botten
<Response [200]>
Hip-Hop/Rap
98
juice+wrld+hate+the+other+side+
<Response [200]>
Hip-Hop/Rap
99
dardan+h+<3+t+e+l
<Response [200]>
Rap
100
bausa+selfmade+babylon+
<Response [200]>
regard+ride+it
<Response [200]>
Dance
102
the+cratez+skifahren+

jamule+medmen
<Response [200]>
Hip-Hop/Rap
45
kayef+ich+würd+lügen
<Response [200]>
Pop
46
rosalía+tkn+
<Response [200]>
Pop
47
juri+death+row
<Response [200]>
Hip-Hop/Rap
48
ak+ausserkontrolle+gangster+queen
<Response [200]>
juice+wrld+wishing+well
<Response [200]>
Hip-Hop/Rap
50
bozza+unbekannt
<Response [200]>
Hip-Hop/Rap
51
lady+gaga+rain+on+me+
<Response [200]>
Pop
52
nico+santos+like+i+love+you
<Response [200]>
Pop
53
jerome+take+my+hand
<Response [200]>
Electronica
54
reezy+sandmann
<Response [200]>
Hip-Hop/Rap
55
zoe+wees+control
<Response [200]>
Pop
56
drake+toosie+slide
<Response [200]>
Hip-Hop/Rap
57
kc+rebell+valla+nein!+
<Response [200]>
Hip-Hop/Rap
58
azet+sin+city
<Response [200]>
Hip-Hop/Rap
59
felix+jaehn+sicko
<Response [200]>
Dance
60
david+puentez+superstar
<Response [200]>
Dance
61
pop+smoke+the+woo+
<Response [200]>
Hip-Hop/Rap
62
jason+derulo+coño
<Response [200]>
Dance
63
lum!x+the+passenger+
<Response [200]>
Dance
64
mozzik+baby
<Response [200]>
Hip-Hop/Rap
65


pashanim+airwaves
<Response [200]>
Hip-Hop/Rap
8
azet+zwei
<Response [200]>
Hip-Hop/Rap
9
dardan+ma+bae
<Response [200]>
pietro+lombardi+cinderella
<Response [200]>
Pop
11
ak+ausserkontrolle+in+meinem+benz
<Response [200]>
vize+never+let+me+down
<Response [200]>
Dance
13
kontra+k+tiefschwarz+
<Response [200]>
Hip-Hop/Rap
14
robin+schulz+alane
<Response [200]>
Dance
15
luciano+halb+3+
<Response [200]>
Hip-Hop/Rap
16
miksu+/+macloud+xxl
<Response [200]>
Hip-Hop/Rap
17
kitschkrieg+oh+junge
<Response [200]>
Hip-Hop/Rap
18
the+weeknd+blinding+lights
<Response [200]>
R&B/Soul
19
twocolors+lovefool
<Response [200]>
Dance
20
fourty+weisser+rauch
<Response [200]>
Hip-Hop/Rap
21
sarhad+sinne
<Response [200]>
Hip-Hop/Rap
22
ufo361+emotions+2.0
<Response [200]>
Hip-Hop/Rap
23
apache+207+boot
<Response [200]>
Hip-Hop/Rap
24
apache+207+fame
<Response [200]>
Hip-Hop/Rap
25
juju+vertrau+mir
<Response [200]>
Hip-Hop/Rap
26
dj+khaled+greece+
<Response [200]>
Hip-Hop/Rap
27
juice+wrld+come+&+go+
<Respons

juju+kein+wort
<Response [200]>
Hip-Hop/Rap
172
lil+lano+ocb
<Response [200]>
Hip-Hop/Rap
173
capital+bra+tilidin
<Response [200]>
Hip-Hop/Rap
174
juri+death+row
<Response [200]>
Hip-Hop/Rap
175
rin+alien
<Response [200]>
Rock
176
bonez+mc+500+ps
<Response [200]>
Hip-Hop/Rap
177
s1mba+rover+
<Response [200]>
Hip-Hop/Rap
178
luciano+sip
<Response [200]>
Reggae
179
shirin+david+90-60-111
<Response [200]>
Hip-Hop/Rap
180
taylor+swift+the+1
<Response [200]>
Soundtrack
181
trevor+daniel+falling
<Response [200]>
Pop
182
juice+wrld+lifes+a+mess+
<Response [200]>
Hip-Hop/Rap
183
rin+keine+liebe
<Response [200]>
Hip-Hop/Rap
184
juice+wrld+lucid+dreams
<Response [200]>
Hip-Hop/Rap
185
r3hab+all+around+the+world+
<Response [200]>
Dance
186
ofenbach+head+shoulders+knees+&+toes+
<Response [200]>
Electronic
187
curtis+waters+stunnin+
<Response [200]>
Alternative
188
sarah+lombardi+te+amo+mi+amor
<Response [200]>
Pop
189
dynoro+in+my+mind
<Response [200]>
Dance
190
kalazh44+dupont
<Response [200]>
Hi

In [ ]:
#Testing on spotify release date.

credentials = SpotifyClientCredentials(client_id="6b8119b6cec94f538151d6bb7fd99acd", client_secret="bb3cc9f8fdb04ddb9516ab01c7bfc257")
token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)
features = spotify.audio_features(str(df['ID'].iloc[0]))
result = spotify.track('5CtI0qwDJkDQGwXD1H1cLb')
print(result)